In [1]:
import sys
sys.path.append("../classes")

In [97]:
import pickle
from functools import reduce

from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA

from statsmodels.sandbox.stats.multicomp import multipletests
from imblearn.over_sampling import RandomOverSampler

import numpy as np, pandas as pd
from geno_classifier import *
from geno_utils import *
from itertools import starmap

from metabolitics.utils import load_network_model
import GEOparse

from cobra import Model

### sample types

01	Primary Solid Tumor	TP<br />
02	Recurrent Solid Tumor	TR<br />
03	Primary Blood Derived Cancer - Peripheral Blood	TB<br />
04	Recurrent Blood Derived Cancer - Bone Marrow	TRBM<br />
05	Additional - New Primary	TAP<br />
06	Metastatic	TM<br />
07	Additional Metastatic	TAM<br />
08	Human Tumor Original Cells	THOC<br />
09	Primary Blood Derived Cancer - Bone Marrow	TBM<br />
10	Blood Derived Normal	NB<br />
11	Solid Tissue Normal	NT<br />
12	Buccal Cell Normal	NBC<br />
13	EBV Immortalized Normal	NEBV<br />
14	Bone Marrow Normal	NBM<br />
15	sample type 15	15SH<br />
16	sample type 16	16SH<br />
20	Control Analyte	CELLC<br />
40	Recurrent Blood Derived Cancer - Peripheral Blood	TRB<br />
50	Cell Lines	CELL<br />
60	Primary Xenograft Tissue	XP<br />
61	Cell Line Derived Xenograft Tissue	XCL<br />
99	sample type 99	99SH<br />

#### https://gdc.cancer.gov/resources-tcga-users/tcga-code-tables/sample-type-codes

In [4]:
healthy_suffix = ('11')
unhealthy_suffix = ('01', '06')

In [5]:
breast_cancer_dataset = pd.read_csv("../datasets/AgilentG4502A_07_3", sep="\t")
breast_cancer_dataset[breast_cancer_dataset.columns[:5]].head()

,sample,TCGA-A8-A0A7-01,TCGA-A8-A07G-01,TCGA-A8-A08R-01,TCGA-A8-A06Y-01
0,RNF14,0.496583,1.004500,-0.149417,1.045250
1,DUOXA1,0.578000,1.695500,0.790000,2.308000
2,UBE2Q2,0.279500,0.795167,0.829667,0.479333
3,RNF10,-0.222250,0.632000,-0.017750,1.457750
4,RNF11,1.131750,0.857750,-0.155500,-0.016500


In [6]:
columns = breast_cancer_dataset.columns

healthy = set(filter(lambda column: column.endswith(healthy_suffix), columns))
unhealthy = set(filter(lambda column: column.endswith(unhealthy_suffix), columns))

### number of samples

In [6]:
print(len(healthy), len(unhealthy))

63 534


In [7]:
healthy & unhealthy

set()

In [20]:
labels_ = [(h, 'healthy') for h in healthy] + [(d, 'unhealthy') for d in unhealthy]

In [ ]:
X, y = parse_database(breast_cancer_dataset, labels=dict(labels_), n_jobs=-1)

### Flux variability analysis

In [26]:
results = pickle.load(open('../results/breast_cancer3.results','rb'))
labels = pickle.load(open('../datasets/breast_cancer3_y','rb'))

X_diff, y = preprocess_results(results, labels, use_diff_score=True, use_pathways=True)

### nested-cross validation over best pipeline (using diff-values)

In [4]:
classifiers = [
    (SVC, {
        'C': np.geomspace(1e-6, 1e6, num=10),
        'class_weight' : ['balanced']
    })
]

feature_selection = [
    (PCA, {
    'n_components': range(3, 81+1, 6)
  })
]

In [5]:
trials, stats = nested_cross_validation(X_diff, y, build_pipelines(feature_selection, classifiers))

Fitting 10 folds for each of 140 candidates, totalling 1400 fits


[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 230 tasks      | elapsed:   52.2s
[Parallel(n_jobs=-1)]: Done 484 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 834 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 1284 tasks      | elapsed: 14.8min
[Parallel(n_jobs=-1)]: Done 1400 out of 1400 | elapsed: 15.2min finished


1 trial done
----------
Fitting 10 folds for each of 140 candidates, totalling 1400 fits


[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 234 tasks      | elapsed:   48.9s
[Parallel(n_jobs=-1)]: Done 486 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 836 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 1286 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 1400 out of 1400 | elapsed: 14.7min finished


2 trial done
----------
Fitting 10 folds for each of 140 candidates, totalling 1400 fits


...

8 trial done
----------
Fitting 10 folds for each of 140 candidates, totalling 1400 fits


[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 231 tasks      | elapsed:   49.3s
[Parallel(n_jobs=-1)]: Done 484 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 834 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 1284 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 1400 out of 1400 | elapsed: 14.7min finished


9 trial done
----------
Fitting 10 folds for each of 140 candidates, totalling 1400 fits


[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed:   47.5s
[Parallel(n_jobs=-1)]: Done 483 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 834 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 1284 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 1400 out of 1400 | elapsed: 14.8min finished


10 trial done
----------


In [6]:
stats.describe()

,fit_time,score_time,test_accuracy,test_f1,test_precision,test_recall
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.027360,0.005827,0.907263,0.947750,0.952113,0.944375
std,0.001517,0.001068,0.005368,0.003069,0.002683,0.004738
min,0.025272,0.004658,0.901254,0.944038,0.948457,0.936373
25%,0.026450,0.004967,0.902843,0.945329,0.950382,0.941929
50%,0.026987,0.005595,0.906241,0.947312,0.951827,0.944654
75%,0.028754,0.006378,0.912138,0.950291,0.953689,0.945790
max,0.029552,0.007678,0.914785,0.952230,0.957194,0.953075


In [40]:
df, _ = preprocess_results(results, labels, *[False, False, False, False])
kbest = SelectKBest().fit(df, _)

In [42]:
df.head()

,10FTHF5GLUtl_max,10FTHF5GLUtl_min,10FTHF5GLUtm_max,10FTHF5GLUtm_min,10FTHF6GLUtl_max,10FTHF6GLUtl_min,10FTHF6GLUtm_max,10FTHF6GLUtm_min,10FTHF7GLUtl_max,10FTHF7GLUtl_min,...,sink_decdicoa_LPAREN_c_RPAREN__max,sink_decdicoa_LPAREN_c_RPAREN__min,sink_octdececoa_LPAREN_c_RPAREN__max,sink_octdececoa_LPAREN_c_RPAREN__min,sink_pre_prot_LPAREN_r_RPAREN__max,sink_pre_prot_LPAREN_r_RPAREN__min,sink_tetdece1coa_LPAREN_c_RPAREN__max,sink_tetdece1coa_LPAREN_c_RPAREN__min,xmpt_max,xmpt_min
unhealthy,2.258389e-10,-3.223306e-10,1.945750e-10,-3.223306e-10,5.442189e-10,-1.687397e-10,2.690967e-10,-1.687397e-10,2.690967e-10,0.000000,...,0.000000,-5.579750e-10,2.057050e-09,-1.969511e-09,0.000000,-8.931238e-10,0.000000,-9.362111e-10,-1.000000e+03,-1.000000e+03
unhealthy,2.940882e-10,-5.212542e-10,2.557954e-10,-5.212542e-10,3.021717e-10,0.000000e+00,3.021717e-10,0.000000e+00,3.021717e-10,0.000000,...,0.000000,-7.416077e-10,2.266120e-09,-7.729568e-10,0.000000,-6.806431e-10,0.000000,-1.948450e-09,6.734808e-10,-3.722448e-09
unhealthy,4.065521e-06,4.065521e-06,4.065521e-06,4.065521e-06,4.065521e-06,4.065521e-06,4.065521e-06,4.065521e-06,4.065521e-06,0.000004,...,0.000004,4.065521e-06,4.065521e-06,4.065521e-06,0.000004,4.065521e-06,0.000004,4.065521e-06,4.065521e-06,4.065521e-06
unhealthy,6.002665e-11,-1.020908e-10,4.473577e-11,-1.020908e-10,7.207746e-11,0.000000e+00,7.207746e-11,0.000000e+00,7.207746e-11,0.000000,...,0.000000,-5.343281e-11,2.326033e-10,-4.047251e-10,0.000000,-3.500880e-10,0.000000,-9.083578e-11,-1.000000e+03,-1.000000e+03
unhealthy,2.471907e-05,2.471907e-05,2.471907e-05,2.471907e-05,2.471907e-05,2.471907e-05,2.471907e-05,2.471907e-05,2.471907e-05,0.000025,...,0.000025,2.471907e-05,2.471907e-05,2.471907e-05,0.000025,2.471907e-05,0.000025,2.471907e-05,2.471907e-05,2.471907e-05


### most significant pathways for breast-cancer without diff-values

In [51]:
corrected_p_values = multipletests(kbest.pvalues_, method='Bonferroni')

In [100]:
significant_reactions = df.columns[corrected_p_values[0]]

template = ('_max', ''), ('_min', '')
reactions = set(map(lambda reaction: reduce(lambda a, kv: a.replace(*kv), template, reaction), 
                significant_reactions))

model = load_network_model('recon2')

In [123]:
pathways = set(filter(lambda name : 'Transport' not in name and name != '', 
   [
    model.reactions.get_by_any(reaction)[0].subsystem
    for reaction in reactions 
   ]))

In [130]:
pathways

{'Nucleotide interconversion',
 'Purine synthesis',
 'Pyrimidine synthesis',
 'Vitamin A metabolism'}

In [121]:
metabolitics_pathways = np.array([
'Alanine and aspartate metabolism_dif',
'Arginine and Proline Metabolism_dif',
'Methionine and cysteine metabolism_dif',
'Taurine and hypotaurine metabolism_dif',
'CoA catabolism_dif',
'Fatty acid oxidation_dif',
'Nucleotide interconversion_dif',
'Eicosanoid metabolism_dif',
'Butanoate metabolism_dif',
'Glycolysis/gluconeogenesis_dif',
'Pentose phosphate pathway_dif',
'Urea cycle_dif',
'Glycine, serine, alanine and threonine metabolism_dif',
'Folate metabolism_dif',
'Glutamate metabolism_dif',
'Sphingolipid metabolism_dif',
'Glycerophospholipid metabolism_dif',
'Fatty acid synthesis_dif',
'ROS detoxification_dif',
'Aminosugar metabolism_dif',
'Citric acid cycle_dif',
'CoA synthesis_dif',
'Pyrimidine catabolism_dif',
'Pyruvate metabolism_dif'])

In [124]:
common = [pathway + '_dif' in metabolitics_pathways for pathway in pathways]

np.array(list(pathways))[common]

array(['Nucleotide interconversion'], dtype='<U26')